<a href="https://colab.research.google.com/github/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/img/Banner_Bootcamp.png?raw=true">

#**Projeto Final de Conclusão de Curso**
###Bootcamp Data Science Aplicada 2 by [Alura](https://www.alura.com.br/) 
####Autor: Helton Cordeiro
e-mail: heltoncordeiro@gmail.com

Junho-Agosto/2021.



---

# **Objetivo desse notebook**

Após os dados disponibilizados pelo Hospital Sírio Libânes terem sido tratados em [notebook próprio](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb) e termos trabalhando com alguns modelos de Machine Learning com seus valores default, vamos explorar os hiperparametros nesse notebook. 

Será utilizando um dicionário com o range de valores e parâmetros  que serão passados para o método **RandomizedSearchCV** utilizando a função *executa_modelos_RandomizedSearchCV* que está no arquivo funcoes.py.

A proposta da separação do [notebook de análise](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/Bootcamp_DataScience_projeto_final.ipynb) se deve ao tempo de processamento necessário para testar todos os modelos, além de facilitar futuros ajustes.

---


##O que é um hiperparâmetro?

Os hiperparâmetros contêm os dados que controlam o próprio processo de treinamento.

Seu aplicativo de treinamento lida com três categorias de dados durante o treinamento do modelo:

* Os *dados de entrada*, também chamados de dados de treinamento, formam uma coleção de registros individuais (instâncias) com as características que são importantes para o problema de machine learning. Esses dados configuram o modelo durante o treinamento para fazer predições precisas sobre novas instâncias de dados semelhantes. No entanto, os valores nos dados de entrada nunca se tornam diretamente parte do modelo.

* Os *parâmetros* do modelo são as variáveis que a técnica de machine learning escolhida usa para ajustar os dados. Por exemplo, uma rede neural profunda (DNN, na sigla em inglês) é composta por nós de processamento (neurônios), cada um com uma operação realizada nos dados enquanto eles trafegam pela rede. Quando a DNN é treinada, cada node tem um valor de peso que informa ao modelo o impacto que ele tem na predição final. Esses pesos são um exemplo dos parâmetros do modelo. De muitas formas, esses parâmetros são o modelo. Ou seja, são eles que diferenciam seu modelo específico de outros modelos do mesmo tipo que trabalham com dados semelhantes.

* Os *hiperparâmetros* são variáveis que controlam o próprio processo de treinamento. Por exemplo, faz parte da configuração de uma rede neural profunda decidir quantas camadas ocultas de nós precisam ser usadas entre a camada de entrada e a camada de saída, bem como quantos nós cada camada precisa usar. Essas variáveis não estão diretamente relacionadas aos dados de treinamento. Elas são variáveis de configuração. Os parâmetros mudam durante um job de treinamento, enquanto os hiperparâmetros geralmente permanecem constantes durante um job.


Os parâmetros do modelo são otimizados (ou seja, "ajustados") pelo processo de treinamento. Você executa os dados por meio das operações do modelo, compara a predição resultante com o valor real de cada instância de dados, avalia a precisão e ajusta até encontrar os melhores valores. Os hiperparâmetros são ajustados por meio da execução de todo o job de treinamento, a observação da precisão agregada e o ajuste. Nos dois casos, você está modificando a composição do modelo tentando encontrar a melhor combinação para lidar com o problema.

Fonte: [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)


##Importação de Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import random
import pickle
from scipy.stats import uniform
from scipy.stats import randint

import warnings
warnings.simplefilter(action='ignore')

In [2]:
rstate = 73246
np.random.seed(rstate)

## Importando as funções auxiliares

* Arquivo funcoes.py contém as funções python que serão utilizados pelo projeto.

In [3]:
from funcoes import executa_modelos_RandomizedSearchCV

In [4]:
help(executa_modelos_RandomizedSearchCV)

Help on function executa_modelos_RandomizedSearchCV in module funcoes:

executa_modelos_RandomizedSearchCV(names, models, dados, n_splits, n_repeats, param_distributions, n_iter, showMsg=True)
    Função que recebe parâmetros para a execução de teste de hiperparametros dos modelos.
    Utiliza a função roda_modelo_RandomizedSearchCV()
    
    Parâmetros
    ----------
      names: array com os nomes dos modelos. É usado como o index do dataFrame de retorno.
               ex.: names = [ "KNeighbors", "Gaussian" ]
      models: array com a instância do modelo a ser testado.
               ex.: classes = [ KNeighborsClassifier(), 
                                GaussianProcessClassifier() ]
      dados: dataFrame com os dados
      n_splits: parâmetro utilizado pelo roda_modelo_RandomizedSearchCV()
      n_repeats: parâmetro utilizado pelo roda_modelo_RandomizedSearchCV()
      param_distributions: dicionário com parâmetros a serem testados
              ex: hiperparams = { 
          

## Carregando os dados
Vamos carregar o arquivo de dados tratados e sem as colunas com alta correção.

O processo de tratamento dos dados estão nesse notebook [projeto_final_tratamento_dados.ipynb](https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/projeto_final_tratamento_dados.ipynb).


### Realizando a carga de dados

In [5]:
url='https://github.com/hc0rd31r0/Bootcamp_Data_Science/blob/main/projeto-final/dados/Kaggle_Sirio_Libanes_ICU_Prediction-tratado-sem-corr.xls?raw=true'
dados_raw = pd.read_excel(url, index_col=0)
dados = dados_raw.copy()



---


#Modelos de Machine Learning
Relação dos Modelos que serão testados nesse projeto, foram selecionados por resolverem problemas de classificação binária e aprendizado supervisionado.


1. [KNeighborsClassifier](
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

2. [SVC - Linear](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

3. [SVC - RBF](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

4. [GaussianProcessClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html)

5. [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)

6. [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

7. [Neural Net (MLP)](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

8. [AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

9. [GradientBoostingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

10. [ExtraTreesClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html)

11. [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)


---



##Modelos

Criando as instâncias dos modelos para processamento.

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression

names = ["KNeighbors", "LinearSVC", "RBFSVC", "Gaussian",
         "DecisionTree", "RandomForest", "NeuralMLP", "AdaBoost",
         "Gradient", "ExtraTrees", "LogisticRegression"]

classes = [
    KNeighborsClassifier(),
    SVC(kernel="linear", probability=True, random_state=rstate),
    SVC(kernel="rbf", probability=True, random_state=rstate),
    GaussianProcessClassifier(random_state=rstate, warm_start=True),
    DecisionTreeClassifier(random_state=rstate),
    RandomForestClassifier(random_state=rstate, bootstrap=False, warm_start=True),
    MLPClassifier(random_state=rstate, warm_start=True),
    AdaBoostClassifier(random_state=rstate),
    GradientBoostingClassifier(random_state=rstate, warm_start=True),
    ExtraTreesClassifier(random_state=rstate, bootstrap=False, warm_start=True),
    LogisticRegression(random_state=rstate, warm_start=True) ]

##Testando Hiperparâmetros

Definindo o dicionário de parâmetros a serem testados.

In [7]:
hiperparams = { 
    "KNeighbors" : { 
        "n_neighbors" : randint(3, 25),
        "weights" : ["uniform", "distance"],
        "leaf_size" : randint(30, 300)
    },
    "LinearSVC" : { 
        "C": uniform(1.0, 5.0),
        "gamma" : ['scale', 'auto'],
        "tol": uniform(0.001,1)
    },
    "RBFSVC" : { 
        "C": uniform(1.0, 5.0),
        "tol": uniform(0.001,1)
    },
    "Gaussian" : {
        "n_restarts_optimizer" : randint(0, 5),
        "max_iter_predict" : randint(100,1000),
    },
    "DecisionTree" : {
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(1, 100),
        "min_samples_split" : randint(2, 200),
        "min_samples_leaf" : randint(1, 200),
        "ccp_alpha" : uniform(0.0, 0.5)
    },
    "RandomForest" : {
        "n_estimators" :randint(90, 500),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(3, 100),
        "min_samples_split" : randint(2, 200),
        "min_samples_leaf" : randint(1, 200),
        "ccp_alpha" : uniform(0.0, 0.5)
    },
    "NeuralMLP" : {
        "activation" : ["identity", "logistic", "tanh", "relu"],
        "solver" : ["lbfgs", "sgd", "adam"],
        "alpha" : uniform(0.0001, 0.1),
        "learning_rate" : ["constant", "invscaling", "adaptive"],
        "learning_rate_init" : uniform(0.001, 0.1),
        "tol": uniform(0.0001,0.01)
    },
    "AdaBoost" : {
        "n_estimators" : randint(50, 1000),
        "learning_rate": uniform(0.5,2),
    },
    "Gradient" : {
        "learning_rate": uniform(0.5,2),
        "n_estimators" : randint(100, 1000),
        "subsample": uniform(0.5,2),
        "max_depth" : randint(3,10),
        "min_samples_split" : randint(2, 200),
        "min_samples_leaf" : randint(1, 200),
        "criterion" : ["friedman_mse", "mse", "mae"]
    },
    "ExtraTrees" : {
        "n_estimators" :randint(100, 1000),
        "criterion" : ["gini", "entropy"],
        "max_depth" : randint(1, 20),
        "min_samples_split" : randint(2, 200),
        "min_samples_leaf" : randint(1, 200),
    },
    "LogisticRegression" : {
        "C": uniform(0.1,2),
        "tol": uniform(0.0001,0.1),
        "solver"   : ["newton-cg", "lbfgs", "liblinear"], 
        "max_iter" : randint(100,1000)
    }
}

##Vamos processar as modelos utilizando 5 splits de dados, com 10 repetições e 15 iterações.

In [8]:
dfmodelosRand = executa_modelos_RandomizedSearchCV(names, classes, dados, 5, 10, hiperparams, 20)

Modelo: KNeighbors 	 tempo: 14 segundos
Modelo: LinearSVC 	 tempo: 32 segundos
Modelo: RBFSVC 	 tempo: 32 segundos
Modelo: Gaussian 	 tempo: 28 segundos
Modelo: DecisionTree 	 tempo: 6 segundos
Modelo: RandomForest 	 tempo: 304 segundos
Modelo: NeuralMLP 	 tempo: 175 segundos
Modelo: AdaBoost 	 tempo: 1150 segundos
Modelo: Gradient 	 tempo: 174 segundos
Modelo: ExtraTrees 	 tempo: 587 segundos
Modelo: LogisticRegression 	 tempo: 15 segundos


In [9]:
dfmodelosRand

,Modelo,AUC,Train AUC,Std AUC,Best Params,Tempo,objRandomizedSearchCV
Nome,,,,,,,
RandomForest,(DecisionTreeClassifier(ccp_alpha=0.0373841499...,0.755744,0.877571,0.059931,"{'ccp_alpha': 0.03738414992391281, 'criterion'...",304,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
LogisticRegression,"LogisticRegression(C=0.39035988101187946, clas...",0.750424,0.827172,0.059262,"{'C': 0.39035988101187946, 'max_iter': 132, 's...",15,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
ExtraTrees,"(ExtraTreeClassifier(ccp_alpha=0.0, class_weig...",0.749170,0.864877,0.066402,"{'criterion': 'gini', 'max_depth': 17, 'min_sa...",587,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
NeuralMLP,"MLPClassifier(activation='identity', alpha=0.0...",0.743306,0.806938,0.063145,"{'activation': 'identity', 'alpha': 0.06470422...",175,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
LinearSVC,"SVC(C=2.79809658001685, break_ties=False, cach...",0.737923,0.855881,0.059068,"{'C': 2.79809658001685, 'gamma': 'scale', 'tol...",32,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
RBFSVC,"SVC(C=4.174218068363189, break_ties=False, cac...",0.735420,0.825203,0.059853,"{'C': 4.174218068363189, 'tol': 0.062447049255...",32,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
AdaBoost,"(DecisionTreeClassifier(ccp_alpha=0.0, class_w...",0.725238,1.000000,0.066237,"{'learning_rate': 1.8575033018752827, 'n_estim...",1150,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
Gaussian,"GaussianProcessClassifier(copy_X_train=True, k...",0.714948,0.950495,0.066657,"{'max_iter_predict': 199, 'n_restarts_optimize...",28,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...
KNeighbors,"KNeighborsClassifier(algorithm='auto', leaf_si...",0.697157,0.745967,0.063374,"{'leaf_size': 39, 'n_neighbors': 24, 'weights'...",14,RandomizedSearchCV(cv=RepeatedStratifiedKFold(...


Na coluna 'Modelo' temos o resultado do best_estimator_, e no campo 'objRandomizedSearchCV' temos uma cópia do objeto resultante do processamento, o retorno da função **RandomizedSearchCV**.

In [17]:
# O Tempo do processamento está em segundos.
dfmodelosRand['Tempo'].sum() / 60

41.95

# Lista dos melhores parâmetros

Após o processamentos temos a lista dos melhores modelos e parâmetros de acordo com as configurações apresentadas no dicionário ```hiperparams```.

In [11]:
for indice in names:
   display(indice, dfmodelosRand.at[indice,'Best Params'])

'KNeighbors'

{'leaf_size': 39, 'n_neighbors': 24, 'weights': 'uniform'}

'LinearSVC'

{'C': 2.79809658001685, 'gamma': 'scale', 'tol': 0.8526212658377876}

'RBFSVC'

...

{'criterion': 'mae',
 'learning_rate': 1.1765927420327074,
 'max_depth': 6,
 'min_samples_leaf': 83,
 'min_samples_split': 21,
 'n_estimators': 773,
 'subsample': 0.9940025668514867}

'ExtraTrees'

{'criterion': 'gini',
 'max_depth': 17,
 'min_samples_leaf': 5,
 'min_samples_split': 72,
 'n_estimators': 874}

'LogisticRegression'

{'C': 0.39035988101187946,
 'max_iter': 132,
 'solver': 'lbfgs',
 'tol': 0.009230202361423178}

In [12]:
for indice in names:
   display(indice, dfmodelosRand.at[indice,'Modelo'])

'KNeighbors'

KNeighborsClassifier(algorithm='auto', leaf_size=39, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=24, p=2,
                     weights='uniform')

'LinearSVC'

SVC(C=2.79809658001685, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3, gamma='scale',
    kernel='linear', max_iter=-1, probability=True, random_state=73246,
    shrinking=True, tol=0.8526212658377876, verbose=False)

'RBFSVC'

...

GradientBoostingClassifier(ccp_alpha=0.0, criterion='mae', init=None,
                           learning_rate=1.1765927420327074, loss='deviance',
                           max_depth=6, max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=83, min_samples_split=21,
                           min_weight_fraction_leaf=0.0, n_estimators=773,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=73246, subsample=0.9940025668514867,
                           tol=0.0001, validation_fraction=0.1, verbose=0,
                           warm_start=True)

'ExtraTrees'

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=17, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=5, min_samples_split=72,
                     min_weight_fraction_leaf=0.0, n_estimators=874,
                     n_jobs=None, oob_score=False, random_state=73246,
                     verbose=0, warm_start=True)

'LogisticRegression'

LogisticRegression(C=0.39035988101187946, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=132, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=73246, solver='lbfgs', tol=0.009230202361423178,
                   verbose=0, warm_start=True)

#Salvando nosso dataFrame

In [15]:
output = open('/content/drive/MyDrive/databases/dfmodelosHP', 'wb')
pickle.dump(dfmodelosRand, output)
output.close()

---

## Referências

* [Kaggle - COVID-19 - Clinical Data to assess diagnosis - Sírio Libanês](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)
* [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html)
* [SciKit Learn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)
* [Configurar o treinamento do AutoML com Python](https://docs.microsoft.com/pt-br/azure/machine-learning/how-to-configure-auto-train)
* [Visão geral do ajuste de hiperparâmetros](https://cloud.google.com/ai-platform/training/docs/hyperparameter-tuning-overview?hl=pt-br)
* [Sklearn.RandomizedSearchCV()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)